<a href="https://colab.research.google.com/github/suzzini/DS_Study/blob/main/%EA%B8%B0%EC%A1%B4_feature%EB%A1%9C_%EC%83%88%EB%A1%9C%EC%9A%B4_feature_%EC%83%9D%EC%84%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
url = 'https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/nba/nba_kobe_bryant.csv'
df = pd.read_csv(url)
df

,game_id,game_event_id,player_name,period,minutes_remaining,seconds_remaining,action_type,shot_type,shot_zone_basic,shot_zone_area,shot_zone_range,shot_distance,loc_x,loc_y,shot_made_flag,game_date,htm,vtm,season_type
0,29600027,102,Kobe Bryant,1,0,42,Jump Shot,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,18,-140,116,0,1996-11-03,LAL,MIN,Regular Season
1,29600031,127,Kobe Bryant,2,10,8,Jump Shot,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,16,-131,97,0,1996-11-05,NYK,LAL,Regular Season
2,29600044,124,Kobe Bryant,2,8,37,Jump Shot,3PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,23,-142,181,1,1996-11-06,CHH,LAL,Regular Season
3,29600044,144,Kobe Bryant,2,6,34,Jump Shot,3PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,0,0,0,0,1996-11-06,CHH,LAL,Regular Season
4,29600044,151,Kobe Bryant,2,5,27,Jump Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),8-16 ft.,13,-10,138,1,1996-11-06,CHH,LAL,Regular Season
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30692,21501228,272,Kobe Bryant,3,9,45,Jump Shot,3PT Field Goal,Above the Break 3,Left Side Center(LC),24+ ft.,25,-96,237,0,2016-04-13,LAL,UTA,Regular Season
30693,21501228,274,Kobe Bryant,3,9,40,Driving Layup Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,0,-4,0,1,2016-04-13,LAL,UTA,Regular Season
30694,21501228,281,Kobe Bryant,3,9,0,Jump Shot,3PT Field Goal,Above the Break 3,Center(C),24+ ft.,31,-32,313,0,2016-04-13,LAL,UTA,Regular Season
30695,21501228,200,Kobe Bryant,2,4,56,Driving Layup Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,0,6,-5,1,2016-04-13,LAL,UTA,Regular Season


## 기존 feature을 가지고 새로운 feature 생성하기

농구 경기의 상대팀을 가르기는 column생성

- htm이 LAL이면 상대팀은 vtm열에 나와있는 팀
- htm이 LAL이 아니면, 상대팀은 htm열에 나와있는 팀

위에 도메인을 가지고 opponent열을 생성할꺼임

In [3]:
df['opp']=0

for i in df.index:
  if (df['htm'][i]=='LAL'):
    df['opp'][i]=df['vtm'][i]
  else:
    df['opp'][i]=df['htm'][i]

df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,game_id,game_event_id,player_name,period,minutes_remaining,seconds_remaining,action_type,shot_type,shot_zone_basic,shot_zone_area,shot_zone_range,shot_distance,loc_x,loc_y,shot_made_flag,game_date,htm,vtm,season_type,opp
0,29600027,102,Kobe Bryant,1,0,42,Jump Shot,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,18,-140,116,0,1996-11-03,LAL,MIN,Regular Season,MIN
1,29600031,127,Kobe Bryant,2,10,8,Jump Shot,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,16,-131,97,0,1996-11-05,NYK,LAL,Regular Season,NYK
2,29600044,124,Kobe Bryant,2,8,37,Jump Shot,3PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,23,-142,181,1,1996-11-06,CHH,LAL,Regular Season,CHH
3,29600044,144,Kobe Bryant,2,6,34,Jump Shot,3PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,0,0,0,0,1996-11-06,CHH,LAL,Regular Season,CHH
4,29600044,151,Kobe Bryant,2,5,27,Jump Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),8-16 ft.,13,-10,138,1,1996-11-06,CHH,LAL,Regular Season,CHH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30692,21501228,272,Kobe Bryant,3,9,45,Jump Shot,3PT Field Goal,Above the Break 3,Left Side Center(LC),24+ ft.,25,-96,237,0,2016-04-13,LAL,UTA,Regular Season,UTA
30693,21501228,274,Kobe Bryant,3,9,40,Driving Layup Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,0,-4,0,1,2016-04-13,LAL,UTA,Regular Season,UTA
30694,21501228,281,Kobe Bryant,3,9,0,Jump Shot,3PT Field Goal,Above the Break 3,Center(C),24+ ft.,31,-32,313,0,2016-04-13,LAL,UTA,Regular Season,UTA
30695,21501228,200,Kobe Bryant,2,4,56,Driving Layup Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,0,6,-5,1,2016-04-13,LAL,UTA,Regular Season,UTA


위의 코드로 생성 완료


별 것도 아닌데 한참 고민했다 ; ㅠ

### 레퍼런스 코드 참고

- Homecourt Advantage: 홈 팀(htm)이 Los Angeles Lakers (LAL)인지 여부
- Opponent: LA Lakers를 상대하는 팀을 나타내는 feature
- Seconds remaining in the period: 피리어드에 남은 총 시간 (분과 초를 합칩니다)
- Seconds remaining in the game: 게임에 남은 총 시간 (피리어드와 초를 합칩니다). 농구에서 게임은 총 4개의 피리어드로 이루어져 있습니다 (각 12분)
- Made previous shot: 코비 브라이언트의 예전 슛이 성공했는지 여부

In [5]:
df.head()

,game_id,game_event_id,player_name,period,minutes_remaining,seconds_remaining,action_type,shot_type,shot_zone_basic,shot_zone_area,shot_zone_range,shot_distance,loc_x,loc_y,shot_made_flag,game_date,htm,vtm,season_type,opp
0,29600027,102,Kobe Bryant,1,0,42,Jump Shot,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,18,-140,116,0,1996-11-03,LAL,MIN,Regular Season,MIN
1,29600031,127,Kobe Bryant,2,10,8,Jump Shot,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,16,-131,97,0,1996-11-05,NYK,LAL,Regular Season,NYK
2,29600044,124,Kobe Bryant,2,8,37,Jump Shot,3PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,23,-142,181,1,1996-11-06,CHH,LAL,Regular Season,CHH
3,29600044,144,Kobe Bryant,2,6,34,Jump Shot,3PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,0,0,0,0,1996-11-06,CHH,LAL,Regular Season,CHH
4,29600044,151,Kobe Bryant,2,5,27,Jump Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),8-16 ft.,13,-10,138,1,1996-11-06,CHH,LAL,Regular Season,CHH


In [6]:
df['htm'].apply(lambda x: 1 if x == 'LAL' else 0)

0        1
1        0
2        0
3        0
4        0
        ..
30692    1
30693    1
30694    1
30695    1
30696    1
Name: htm, Length: 30697, dtype: int64

In [7]:
df['game_id'].iloc[1]

29600031

In [9]:
def make_feature(df):

    df_tmp = df.copy()

    # Homecourt Advantage
    df_tmp['homecourt_advantage'] = df_tmp['htm'].apply(lambda x: 1 if x == 'LAL' else 0)
    
    # Opponent
    df_tmp['opponent'] = (df_tmp['htm'] + df_tmp['vtm']).str.replace('LAL','')
    
    # Seconds remaining in the period
    df_tmp['seconds_remaining_in_the_period'] = (df_tmp['minutes_remaining']*60) + df['seconds_remaining']
    
    # Seconds remaining in the game
    df_tmp['seconds_remaining_in_the_game'] = (df_tmp['period']*(12*60)) + (df_tmp['minutes_remaining']*60) + df_tmp['seconds_remaining']
    
    # Made previous shot
    df_tmp['made_previous_shot'] = ''
    for i in range(len(df_tmp)):
        if i == 0:
            df_tmp['made_previous_shot'].iloc[i] = df_tmp['shot_made_flag'].iloc[i]
        else:
            df_tmp['made_previous_shot'].iloc[i] = df_tmp['shot_made_flag'].iloc[i-1]

    return df_tmp

make_feature(df)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,game_id,game_event_id,player_name,period,minutes_remaining,seconds_remaining,action_type,shot_type,shot_zone_basic,shot_zone_area,shot_zone_range,shot_distance,loc_x,loc_y,shot_made_flag,game_date,htm,vtm,season_type,opp,homecourt_advantage,opponent,seconds_remaining_in_the_period,seconds_remaining_in_the_game,made_previous_shot
0,29600027,102,Kobe Bryant,1,0,42,Jump Shot,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,18,-140,116,0,1996-11-03,LAL,MIN,Regular Season,MIN,1,MIN,42,762,0
1,29600031,127,Kobe Bryant,2,10,8,Jump Shot,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,16,-131,97,0,1996-11-05,NYK,LAL,Regular Season,NYK,0,NYK,608,2048,0
2,29600044,124,Kobe Bryant,2,8,37,Jump Shot,3PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,23,-142,181,1,1996-11-06,CHH,LAL,Regular Season,CHH,0,CHH,517,1957,0
3,29600044,144,Kobe Bryant,2,6,34,Jump Shot,3PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,0,0,0,0,1996-11-06,CHH,LAL,Regular Season,CHH,0,CHH,394,1834,1
4,29600044,151,Kobe Bryant,2,5,27,Jump Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),8-16 ft.,13,-10,138,1,1996-11-06,CHH,LAL,Regular Season,CHH,0,CHH,327,1767,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30692,21501228,272,Kobe Bryant,3,9,45,Jump Shot,3PT Field Goal,Above the Break 3,Left Side Center(LC),24+ ft.,25,-96,237,0,2016-04-13,LAL,UTA,Regular Season,UTA,1,UTA,585,2745,1
30693,21501228,274,Kobe Bryant,3,9,40,Driving Layup Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,0,-4,0,1,2016-04-13,LAL,UTA,Regular Season,UTA,1,UTA,580,2740,0
30694,21501228,281,Kobe Bryant,3,9,0,Jump Shot,3PT Field Goal,Above the Break 3,Center(C),24+ ft.,31,-32,313,0,2016-04-13,LAL,UTA,Regular Season,UTA,1,UTA,540,2700,1
30695,21501228,200,Kobe Bryant,2,4,56,Driving Layup Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,0,6,-5,1,2016-04-13,LAL,UTA,Regular Season,UTA,1,UTA,296,1736,0
